In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("Uber_Customer_Support_Tickets_Anomalous.csv")


In [3]:
print(df.shape)
print(df.info())
print(df.head())

(1001, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticket_ID                1001 non-null   object 
 1   Created_At               1001 non-null   object 
 2   Channel                  1000 non-null   object 
 3   Response_Time_Minutes    1000 non-null   float64
 4   Resolution_Time_Minutes  1001 non-null   int64  
 5   CSAT_Score               1000 non-null   float64
 6   Issue_Type               1000 non-null   object 
 7   Daily_Ticket_Count       1001 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 62.7+ KB
None
  Ticket_ID  Created_At    Channel  Response_Time_Minutes  \
0     T0001  21-02-2025  LiveAgent                    4.0   
1     T0002  15-01-2025  LiveAgent                    7.0   
2     T0003  13-03-2025  LiveAgent                    8.0   
3     T0004  02-03-2025  LiveAge

In [4]:
print("\nMissing Values:\n", df.isnull().sum())


Missing Values:
 Ticket_ID                  0
Created_At                 0
Channel                    1
Response_Time_Minutes      1
Resolution_Time_Minutes    0
CSAT_Score                 1
Issue_Type                 1
Daily_Ticket_Count         0
dtype: int64


#handling missing values:-

In [5]:
df["CSAT_Score"] = df["CSAT_Score"].fillna(df["CSAT_Score"].median())
df["Issue_Type"] = df["Issue_Type"].fillna(df["Issue_Type"].mode()[0])
df["Channel"] = df["Channel"].fillna("Unknown")
df["Response_Time_Minutes"] = df["Response_Time_Minutes"].fillna(df["Response_Time_Minutes"].median())

In [6]:
print("\nMissing Values after cleaning:\n", df.isnull().sum())


Missing Values after cleaning:
 Ticket_ID                  0
Created_At                 0
Channel                    0
Response_Time_Minutes      0
Resolution_Time_Minutes    0
CSAT_Score                 0
Issue_Type                 0
Daily_Ticket_Count         0
dtype: int64


#Standardizing Categorical Columns (Channel, Issue Type):-

In [7]:
df["Channel"] = df["Channel"].str.strip().str.lower()   # remove spaces, lowercase
df["Channel"] = df["Channel"].replace({
    "chat bot": "chatbot",
    "chatbot": "chatbot",
    "liveagent": "live agent",
    "phone": "phone",
    "unknown": "unknown"
})
df["Channel"] = df["Channel"].str.title() 

In [8]:
print("Unique Channels after cleaning:", df["Channel"].unique())

Unique Channels after cleaning: ['Live Agent' 'Chatbot' 'Unknown']


In [9]:
df["Issue_Type"] = df["Issue_Type"].str.strip().str.lower()

df["Issue_Type"] = df["Issue_Type"].replace({
    "lost item": "lost item",
    "lostitem": "lost item",
    "ride cancellation": "ride cancellation",
    "account access": "account access",
    "payment issue": "payment issue"
})
df["Issue_Type"] = df["Issue_Type"].str.title() 

In [10]:
print("Unique Issue Types after cleaning:", df["Issue_Type"].unique())

Unique Issue Types after cleaning: ['Ride Cancellation' 'Lost Item' 'Account Access' 'Driver Issue'
 'Payment Problem' 'App Crash']


#Fixing outliers :-

In [11]:
outliers = {}

# Response Time
outliers["Response_Time_Minutes"] = df[(df["Response_Time_Minutes"] < 0) | (df["Response_Time_Minutes"] > 120)]

# Resolution Time
outliers["Resolution_Time_Minutes"] = df[(df["Resolution_Time_Minutes"] < 0) | (df["Resolution_Time_Minutes"] > 200)]

# CSAT Score
outliers["CSAT_Score"] = df[(df["CSAT_Score"] < 1) | (df["CSAT_Score"] > 5)]

# Daily Ticket Count
outliers["Daily_Ticket_Count"] = df[(df["Daily_Ticket_Count"] < 1) | (df["Daily_Ticket_Count"] > 50)]

# Print summary
for col, data_out in outliers.items():
    print(f"\n--- {col} ---")
    print(f"Number of Outliers: {len(data_out)}")
    if not data_out.empty:
        print(data_out.head())



--- Response_Time_Minutes ---
Number of Outliers: 1
   Ticket_ID  Created_At  Channel  Response_Time_Minutes  \
40     T0041  03-01-2025  Chatbot                 9999.0   

    Resolution_Time_Minutes  CSAT_Score         Issue_Type  Daily_Ticket_Count  
40                       50         5.0  Ride Cancellation                  14  

--- Resolution_Time_Minutes ---
Number of Outliers: 1
   Ticket_ID  Created_At  Channel  Response_Time_Minutes  \
45     T0046  08-02-2025  Chatbot                   21.0   

    Resolution_Time_Minutes  CSAT_Score Issue_Type  Daily_Ticket_Count  
45                       -5         3.0  Lost Item                  12  

--- CSAT_Score ---
Number of Outliers: 0

--- Daily_Ticket_Count ---
Number of Outliers: 0


In [28]:
# Cap resolution times at 200 mins
df.loc[df["Resolution_Time_Minutes"] < 0, "Resolution_Time_Minutes"] = 0
df.loc[df["Resolution_Time_Minutes"] > 200, "Resolution_Time_Minutes"] = 200
#fix response time
df.loc[df["Response_Time_Minutes"] < 0, "Response_Time_Minutes"] = 0
df.loc[df["Response_Time_Minutes"] > 120, "Response_Time_Minutes"] = 120

#Converting Dates:- 

In [29]:
df["Created_At"] = pd.to_datetime(df["Created_At"], format="%d-%m-%Y", errors="coerce")

#removing duplicates

In [30]:
df = df.drop_duplicates(subset=["Ticket_ID"], keep="first")

#Handling Numeric Values

In [31]:
df["Response_Time_Minutes"] = pd.to_numeric(df["Response_Time_Minutes"], errors="coerce")


In [32]:
df["Response_Time_Minutes"] = df["Response_Time_Minutes"].fillna(df["Response_Time_Minutes"].median())


#Summary Of Distribution

In [33]:
print("\nSummary Statistics:")
print("Average CSAT Score:", df["CSAT_Score"].mean())
print("Average Resolution Time:", df["Resolution_Time_Minutes"].mean())
print("Average Daily Ticket Count:", df["Daily_Ticket_Count"].mean())

print("\nData Cleaning & Exploration Completed.")


Summary Statistics:
Average CSAT Score: 3.686
Average Resolution Time: 96.099
Average Daily Ticket Count: 12.212

Data Cleaning & Exploration Completed.


In [34]:
df.to_csv("Uber_Customer_Support_Tickets_Cleaned.csv", index=False)